In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install datasets --quiet
! pip install evaluate --quiet
! pip install transformers --quiet
!pip install huggingface_hub --quiet

!pip install accelerate -U --quiet
!pip install transformers[torch] --quiet
!pip install shap --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 503.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 7.2 MB/s eta 0:00:00


### Generate Labels for Data

In [ ]:
import os
import csv

audio_directory = "/content/drive/MyDrive/all_audio"
#text_directory = "/content/drive/MyDrive/all_audio/all_text"
data = []

for filename in os.listdir(audio_directory):
    if filename.endswith(".wav"):
        # Extract label
        label = filename.split("_")[1]
        label = 1 if label == 'lie' else 0

        # Append data to the list
        data.append((filename, label))

# Specify the path for the metadata CSV file
csv_file_path = "/content/drive/MyDrive/all_audio/metadata.csv"

# Write data to the CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["file_name", "label"])
    writer.writerows(data)

print(f"Metadata file created at {csv_file_path}")


Metadata file created at /content/drive/MyDrive/all_audio/metadata.csv


### Split data into sliding windows

In [19]:
# Windowing function
def window_audio(audio_array, window_size=10, overlap=0.75):
    sr = 16000
    window_size_samples = int(window_size * sr)
    overlap_samples = int(window_size_samples * overlap)

    windows = []
    for i in range(0, len(audio_array) - window_size_samples, overlap_samples):
        window = audio_array[i:i + window_size_samples]
        windows.append(window)

    return windows


In [18]:
def get_text_embeddings(texts, tokenizer, model):
    encoded_texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        model_output = model(**encoded_texts)

    text_embeddings = model_output.last_hidden_state[:, 0, :]

    return text_embeddings

In [17]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000*10, truncation=True)
    return inputs

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)

    # Calculate precision, recall, and f1 score
    precision = precision_score(y_true=eval_pred.label_ids, y_pred=predictions, average='weighted')
    recall = recall_score(y_true=eval_pred.label_ids, y_pred=predictions, average='weighted')
    f1 = f1_score(y_true=eval_pred.label_ids, y_pred=predictions, average='weighted')

    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1}

In [6]:
from collections import defaultdict
from datasets import Dataset as DT

def apply_window(dataset):
    windowed_dataset = defaultdict(list)

    for instance_id, example in enumerate(dataset):
        audio_array = example["audio"]["array"]
        windows = window_audio(audio_array)

        for window in windows:
            windowed_dataset["audio"].append({"array": window})
            windowed_dataset["label"].append(example["label"])
            windowed_dataset["instance_id"].append(instance_id)

    windowed_dataset = DT.from_dict(windowed_dataset)
    return windowed_dataset


### Training


In [7]:
from datasets import load_dataset
from datasets import Audio
from datasets import DatasetDict
from datasets import Dataset as DT
from transformers import AutoFeatureExtractor
import evaluate
import numpy as np
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,confusion_matrix,precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from collections import defaultdict
from transformers import EarlyStoppingCallback
import torch

### TO BE RUN AFTER QUESTION PART IS TAKEN OUT FROM THE CUSTOM DATASET.
### BECAUSE WE WONT NEED WINDOWS

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def train_model_custom_data(dataset):
  for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
      print(f"\n----- Fold {fold + 1} -----")

      train_dataset = dataset.select(train_idx)
      test_dataset = dataset.select(test_idx)
      print(train_dataset)
      print(test_dataset)
      train_dataset = apply_window(train_dataset)
      test_dataset = apply_window(test_dataset)
      print(train_dataset)
      print(test_dataset)

      encoded_train_dataset = train_dataset.map(preprocess_function, remove_columns="audio", batched=True)
      encoded_test_dataset = test_dataset.map(preprocess_function, remove_columns="audio", batched=True)

      num_labels = 2
      model = AutoModelForAudioClassification.from_pretrained(
         "facebook/hubert-base-ls960", num_labels=num_labels
      )
      model.to(device)

      training_args = TrainingArguments(
          output_dir= "hubert_deception-1",
          evaluation_strategy="epoch",
          save_strategy="epoch",
          learning_rate=3e-5,
          per_device_train_batch_size=8,
          gradient_accumulation_steps=4,
          per_device_eval_batch_size=8,
          num_train_epochs=10,
          warmup_ratio=0.1,
          logging_steps=10,
          load_best_model_at_end=True,
          push_to_hub=False,
        )

      feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")

      trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_train_dataset,
        eval_dataset=encoded_test_dataset,
        tokenizer=feature_extractor,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(3, 0.0)]
    )

      trainer.train()

      eval_results = trainer.evaluate()


      print(f"Fold {fold + 1} - F1 Score: {eval_results['eval_f1_score']}")
      all_f1_scores.append(eval_results['eval_f1_score'])

  mean_f1_score = np.mean(all_f1_scores)
  print(f"\nMean F1 Score across all folds: {mean_f1_score}")



Using device: cuda


In [8]:
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import torch
from datasets import load_dataset
from datasets import Audio
from datasets import DatasetDict
from datasets import Dataset as DT
from transformers import AutoFeatureExtractor
import evaluate
import numpy as np
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,confusion_matrix,precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from collections import defaultdict
from transformers import EarlyStoppingCallback
import torch

In [20]:
def train_model(dataset):
    for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\n----- Fold {fold + 1} -----")

        train_dataset = dataset.select(train_idx)
        test_dataset = dataset.select(test_idx)
        print(train_dataset)
        print(test_dataset)
        train_dataset = apply_window(train_dataset)
        test_dataset = apply_window(test_dataset)
        print(train_dataset)
        print(test_dataset)

        encoded_train_dataset = train_dataset.map(preprocess_function, remove_columns="audio", batched=True)
        encoded_test_dataset = test_dataset.map(preprocess_function, remove_columns="audio", batched=True)


        num_labels = 2
        model = AutoModelForAudioClassification.from_pretrained(
            "facebook/hubert-base-ls960", num_labels=num_labels
        )
        model.to(device)

        training_args = TrainingArguments(
            output_dir="hubert_deception-1",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=3e-5,
            per_device_train_batch_size=8,
            gradient_accumulation_steps=4,
            per_device_eval_batch_size=8,
            num_train_epochs=10,
            warmup_ratio=0.1,
            logging_steps=10,
            load_best_model_at_end=True,
            push_to_hub=False,
        )

        feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=encoded_train_dataset,
            eval_dataset=encoded_test_dataset,
            tokenizer=feature_extractor,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(3, 0.0)]
        )

        trainer.train()

        eval_results = trainer.evaluate()


        predictions = trainer.predict(encoded_test_dataset)

        # Apply averaging using instance_id
        instance_id_predictions = defaultdict(list)
        for i, instance_id in enumerate(test_dataset["instance_id"]):
            instance_id_predictions[instance_id].append(predictions.predictions[i])


        for instance_id, instance_predictions in instance_id_predictions.items():
            mean_prediction = np.mean(instance_predictions, axis=0)
            eval_results[f'eval_predictions_{instance_id}'] = mean_prediction


        print(f"Fold {fold + 1} - F1 Score: {eval_results['eval_f1_score']}")
        all_f1_scores.append(eval_results['eval_f1_score'])

    mean_f1_score = np.mean(all_f1_scores)
    print(f"\nMean F1 Score across all folds: {mean_f1_score}")


In [13]:
#read data
dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/all_audio",split='train')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
print(dataset)

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
num_labels = 2
kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_f1_scores = []
validation_indices = list(range(15)) + list(range(len(dataset['label']) - 25, 15))
validation_dataset = dataset.select(validation_indices)
dataset = dataset.select([i for i in range(len(dataset)) if i not in validation_indices])

train_model(dataset)

Resolving data files:   0%|          | 0/116 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'label'],
    num_rows: 115
})

----- Fold 1 -----
Dataset({
    features: ['audio', 'label'],
    num_rows: 80
})
Dataset({
    features: ['audio', 'label'],
    num_rows: 20
})
Dataset({
    features: ['audio', 'label', 'instance_id'],
    num_rows: 233
})
Dataset({
    features: ['audio', 'label', 'instance_id'],
    num_rows: 61
})


Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Score
0,No log,0.676697,0.861534,0.641509,0.501410
1,0.690300,0.645466,0.861534,0.641509,0.501410
2,0.669100,0.601098,0.861534,0.641509,0.501410
4,0.580300,0.452929,0.817385,0.811321,0.813179
5,0.475800,0.382960,0.888256,0.867925,0.870362
6,0.349500,0.366340,0.851198,0.830189,0.833323
8,0.302400,0.410573,0.832619,0.830189,0.831098
9,0.253100,0.411121,0.832619,0.830189,0.831098


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 5 - F1 Score: 0.833322885977559

Mean F1 Score across all folds: 0.8280958382053436


# CUSTOM DATASET

In [14]:
dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/RecordingsWav",split='train')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
print(dataset)

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
num_labels = 2
kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_f1_scores = []
validation_indices = list(range(15)) + list(range(len(dataset['label']) - 25, 15))
validation_dataset = dataset.select(validation_indices)
dataset = dataset.select([i for i in range(len(dataset)) if i not in validation_indices])

train_model(dataset)

Resolving data files:   0%|          | 0/121 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['audio', 'label'],
    num_rows: 120
})

----- Fold 1 -----
Dataset({
    features: ['audio', 'label'],
    num_rows: 84
})
Dataset({
    features: ['audio', 'label'],
    num_rows: 21
})
Dataset({
    features: ['audio', 'label', 'instance_id'],
    num_rows: 104
})
Dataset({
    features: ['audio', 'label', 'instance_id'],
    num_rows: 24
})


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Score
0,No log,0.648570,0.774400,0.880000,0.823830
1,No log,0.607086,0.774400,0.880000,0.823830
2,No log,0.555938,0.774400,0.880000,0.823830
4,0.683600,0.537885,0.774400,0.880000,0.823830
5,0.683600,0.537069,0.774400,0.880000,0.823830
6,0.668400,0.536427,0.774400,0.880000,0.823830


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 5 - F1 Score: 0.8238297872340425

Mean F1 Score across all folds: 0.7070543082209164


# Train Model on Bag of Lies Dataset

### Generate Metadata

In [ ]:
import pandas as pd
import re
import os
import csv

annot = pd.read_csv('/content/drive/MyDrive/bag-of-lies/BagOfLies/Annotations.csv')
annot[['video','truth']]
annot['video'] = annot['video'].str.replace("./Finalised/","")
annot['video'] = annot['video'].str.replace("/","_")
annot['video'] = annot['video'].str.replace("_video.mp4","")
annot['video'] = annot['video'].str.replace("_video.mp4","")
def rearrange_string(input_str):
    parts = input_str.split('_')
    rearranged_str = f"{parts[2]}_{parts[3]}_{parts[0]}_{parts[1]}"
    return rearranged_str

for i in range(len(annot['video'])):
  annot['video'][i] = rearrange_string(annot['video'][i])


directory = "/content/drive/MyDrive/bag-of-lies/BagOfLies/Audio"
data = []
for filename in os.listdir(directory):
    if filename.endswith(".wav"):
        user_run = filename.replace(".wav", "")
        label = annot.copy().loc[annot['video'] == user_run]['truth'].reset_index()['truth'][0]
        if label == 1:
          label = 0
        else:
          label = 1
        data.append((filename, label))

csv_file_path = "/content/drive/MyDrive/bag-of-lies/BagOfLies/Audio/metadata.csv"


with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["file_name", "label"])
    writer.writerows(data)

print(f"Metadata file created at {csv_file_path}")



<ipython-input-92-28c6955affde>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  annot['video'] = annot['video'].str.replace("./Finalised/","")
<ipython-input-92-28c6955affde>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  annot['video'] = annot['video'].str.replace("_video.mp4","")
<ipython-input-92-28c6955affde>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot['video'][i] = rearrange_string(annot['video'][i])


Metadata file created at /content/drive/MyDrive/bag-of-lies/BagOfLies/Audio/metadata.csv


### Training

In [ ]:
from datasets import load_dataset
from datasets import Audio
from datasets import DatasetDict, Dataset
from transformers import AutoFeatureExtractor
import evaluate
import numpy as np
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score,confusion_matrix,precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from collections import defaultdict
from transformers import EarlyStoppingCallback

In [16]:
#read data
dataset = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/Audio-BagofLies",split='train')
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
print(dataset)

first_indices = list(range(15))
last_indices = list(range(len(dataset['label']) - 15, len(dataset['label'])))
validation_indices = first_indices + last_indices
validation_dataset = dataset.select(validation_indices)

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
num_labels = 2
kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_f1_scores = []

dataset = dataset.select([i for i in range(len(dataset)) if i not in validation_indices])

train_model(dataset)


Resolving data files:   0%|          | 0/326 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'label', 'text_x', 'text_y'],
    num_rows: 325
})

----- Fold 1 -----
Dataset({
    features: ['audio', 'label', 'text_x', 'text_y'],
    num_rows: 236
})
Dataset({
    features: ['audio', 'label', 'text_x', 'text_y'],
    num_rows: 59
})
Dataset({
    features: ['audio', 'label', 'instance_id'],
    num_rows: 212
})
Dataset({
    features: ['audio', 'label', 'instance_id'],
    num_rows: 60
})


Map:   0%|          | 0/212 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1 Score
1,No log,0.697177,0.619550,0.861765,0.690196
2,0.686600,0.702742,0.619550,0.861765,0.690196
3,0.670300,0.631127,0.608997,0.549020,0.539948
4,0.670300,0.593075,0.664516,0.666667,0.665334
5,0.635000,0.572710,0.786281,0.784314,0.784994
6,0.624300,0.580577,0.723982,0.705882,0.708158
7,0.624300,0.629248,0.714795,0.647059,0.642157
8,0.636900,0.595921,0.695652,0.666667,0.668464


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 5 - F1 Score: 0.7849938556423751

Mean F1 Score across all folds: 0.760467864416674
